In [1]:
import ray
from ray import rllib, tune
import pandas as pd
import numpy as np
from datetime import datetime
import trading_gym
# from trading_gym.registry.gaia.v9.env import GAIAPredictorsContinuousV9
from trading_gym.registry.gaia.v10.env import GAIAPredictorsContinuousV10
from trading_gym.ray.walkforward import WalkForwardRunner, WalkForwardResults
%matplotlib inline
print(trading_gym.__package__, trading_gym.__version__)
print(ray.__package__, ray.__version__)

trading-gym 0.8.1
ray 0.7.2


In [2]:
import trading_gym.registry.gaia.v10

In [3]:
ray.init()

2019-08-23 23:57:29,544	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-23_23-57-29_544115_75101/logs.
2019-08-23 23:57:29,668	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:45683 to respond...
2019-08-23 23:57:29,802	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:23680 to respond...
2019-08-23 23:57:29,808	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-08-23 23:57:29,862	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-23_23-57-29_544115_75101/logs.
2019-08-23 23:57:29,866	WARNING services.py:1298 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-23 23:57:29,867	INFO services.py:1446 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:45683',
 'object_store_address': '/tmp/ray/session_2019-08-23_23-57-29_544115_75101/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-23_23-57-29_544115_75101/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-23_23-57-29_544115_75101'}

## Create the walk-forward partitions
It's responsibility of the user to create the training/test (and maybe validation) partitions to turn the walk forward training. Note that 2-fold split is a particular case of walk-forward training, so you are still able to run a simple 2-fold split.


In [4]:
partitions = list()
for year in range(2007, 2018):
    partition = {
        'training-set': [datetime.min, datetime(year, 12, 31)],
        'test-set': [datetime(year + 1, 1, 1), datetime(year + 1, 12, 31)],
    }
    partitions.append(partition)
partitions


[{'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2007, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2008, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2009, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2010, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2011, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2012, 1, 1, 0, 0),
   datetime.datetime(2012, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 

In [5]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [6]:
# from ray.rllib.models.misc import conv2d as rllibconv2d
# # import tensorflow as tf
# # from tf.nn import conv2d 

# #             Find a way of implementing the activation as an argument 
# #             activation = 'relu' # or tanh or softplus or sigmoid 
# activation = 'relu'

# class CNN(Model):
# #      Check that this is the corect way to use kwargs
#     def _build_layers_v2(self,input_dict:dict, num_outputs: int, config:dict):
#         import tensorflow.contrib.slim as slim
        
#         with tf.name_scope("cnn_net"):
#             last_layer = input_dict['obs']

#             conv_w1 = tf.Variable(tf.truncated_normal([1,3,int(last_layer.shape[3]),2],stddev=0.5),trainable = "True")
#             layer = tf.nn.conv2d(last_layer,filter = conv_w1,padding='VALID',strides= [1,1,1,1])
#             norm = tf.layers.batch_normalization(layer)
#             last_layer = tf.nn.relu(norm)
            
#             con_w2 = tf.Variable(tf.truncated_normal([1,int(last_layer.shape[2]),int(last_layer.shape[3]),48],stddev=0.5),trainable = "True")
#             layer = tf.nn.conv2d(last_layer,filter = conv_w2,padding='VALID',strides=[1,1,1,1])
#             norm = tf.layers.batch_normalization(layer)
#             last_layer = tf.nn.relu(norm)
            
#             con_w3 = tf.Variable(tf.truncated_normal([1,int(last_layer.shape[2]),48,1],stddev=0.5),trainable = "True")
#             layer = tf.nn.conv2d(last_layer,filter = conv_w3, padding='VALID',strides=[1,1,1,1])
#             norm = tf.layers.batch_normalization(layer)
#             last_layer = tf.nn.relu(norm)
            
#             dense_input = last_layer[:,:,0,0]
#             out_dim = num_outputs
# #             Dense fully connected
#             dense_w = tf.Variable(tf.truncated_normal([int(dense_input.shape[1]),out_dim],stddev=0.1,trainable = "True"))
#             dense_b = tf.Variable(tf.constant(0.1,shape[out_dim]), trainable= "True")
#             out = tf.matmul(dense_input,dense_w) + dense_b
            
#             if activation == 'relu':
#                 output = tf.nn.relu(out)
#             elif activation == 'tanh':
#                 output = tf.nn.tanh(out)
#             elif activation == 'softplus':
#                 output = tf.nn.softplus(out)
#             elif activation=='sigmoid':
#                 output = tf.nn.sigmoid(out)
#             else: 
#                 print("CNN Build has failed")
                
        
#         return output, out
# ModelCatalog.register_custom_model(CNN.__name__, CNN)

In [7]:
## Create the config dict
config = ray.rllib.agents.ppo.DEFAULT_CONFIG.copy()
config['env'] = GAIAPredictorsContinuousV10
# config['env_config'] = {
#     'cost_of_commissions': tune.grid_search([0.00005]),
#     'cost_of_spread': 0.0001,
# }
config['gamma'] = tune.grid_search([0.82])  # 2 weeks

In [8]:
config['vf_clip_param'] = 0
config['vf_loss_coeff'] = 0
config['lambda'] = 0
config['use_gae'] = True

# need to have vf share layers if lstm is used
# config['vf_share_layers'] = True
# config['model']['use_lstm'] =  True

config['batch_mode'] = 'complete_episodes'
config['train_batch_size'] = 4000 # tune.grid_search([4000])
config['sgd_minibatch_size'] = 128
config['num_sgd_iter'] = tune.grid_search([8])
config['entropy_coeff'] =  1e-5 # tune.grid_search([1e-5])
config['kl_coeff'] = 0.2 #tune.grid_search([0.2])
config['kl_target'] = tune.grid_search([0.01])
config['clip_param'] = tune.grid_search([0.8])

config['lr'] = tune.grid_search([1e-5])

# config['model']['custom_model'] = CNN.__name__


In [9]:
env = GAIAPredictorsContinuousV10()
env.action_space.sample()

array([-0.07727737,  0.08770195])

In [10]:
## Run your walk-forward experiment
walk_forward = WalkForwardRunner(
    env_partitions=partitions,
    trainable=ray.rllib.agents.ppo.PPOTrainer,
    config=config,
    stop={'timesteps_total': 500000},
    checkpoint_freq=1,
)

Note that WalkForwardRunner has constructed the implied ray Experiment(s) from your walk forward settings.

walk_forward.experiments

Note that trials are associated with a `RestoreID`. This `ID` is all you need to restore an agent. Here we are using a grid search of two values for `cost_of_commissions` on two partitions, so we have a total of 4 experiments.

In [11]:
# trials = tune.run_experiments(walk_forward.experiments, verbose=0)

# NOTE that the code has had to be commented out in order to allow for this 

trials = walk_forward.run(verbose=0)
trials

2019-08-23 23:57:43,009	WARNING experiment.py:218 -- All experiments will be using the same SearchAlgorithm.
2019-08-23 23:57:43,015	INFO tune.py:61 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-08-23 23:57:43,017	INFO tune.py:233 -- Starting a new experiment.
2019-08-23 23:57:43,033	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.


(pid=75451) 2019-08-23 23:57:47,710	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=75451) 2019-08-23 23:57:49,127	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
(pid=75451) 2019-08-23 23:57:49.127868: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=75451) 2019-08-23 23:57:49,380	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:
(pid=75451) 
(pid=75451) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=75451)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=75451)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=75451)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?

(pid=75449) 2019-08-23 23:57:58,594	INFO sample_batch_builder.py:161 -- Trajectory fragment after postprocess_trajectory():
(pid=75449) 
(pid=75449) { 'agent0': { 'data': { 'action_prob': np.ndarray((41,), dtype=float32, min=0.001, max=0.157, mean=0.077),
(pid=75449)                         'actions': np.ndarray((41, 2), dtype=float32, min=-2.357, max=3.116, mean=-0.047),
(pid=75449)                         'advantages': np.ndarray((41,), dtype=float32, min=-0.006, max=0.009, mean=0.0),
(pid=75449)                         'agent_index': np.ndarray((41,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=75449)                         'behaviour_logits': np.ndarray((41, 4), dtype=float32, min=-0.011, max=0.007, mean=-0.0),
(pid=75449)                         'dones': np.ndarray((41,), dtype=bool, min=0.0, max=1.0, mean=0.024),
(pid=75449)                         'eps_id': np.ndarray((41,), dtype=int64, min=328508.0, max=328508.0, mean=328508.0),
(pid=75449)                         'infos':

(pid=75451) 2019-08-23 23:58:12,508	WARNING ppo.py:129 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=75451) 2019-08-23 23:58:13,774	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.


2019-08-23 23:58:16,089	WARNING util.py:64 -- The `process_trial` operation took 0.14609742164611816 seconds to complete, which may be a performance bottleneck.
2019-08-23 23:58:16,203	WARNING util.py:64 -- The `experiment_checkpoint` operation took 0.11031508445739746 seconds to complete, which may be a performance bottleneck.


...

(pid=108860) 2019-08-24 12:25:02,977	WARNING ppo.py:129 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


2019-08-24 12:25:08,401	WARNING util.py:64 -- The `process_trial` operation took 0.13186144828796387 seconds to complete, which may be a performance bottleneck.


(pid=108860) 2019-08-24 12:25:32,999	WARNING ppo.py:129 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


2019-08-24 12:25:40,101	INFO ray_trial_executor.py:187 -- Destroying actor for trial PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-24 12:25:40,126	WARNING util.py:64 -- The `process_trial` operation took 0.21507978439331055 seconds to complete, which may be a performance bottleneck.


[PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV10_restoreID=-1177500123347456453_runID=Dxq7WnXW_0_clip_param=0.8,gamma=0.82,kl_target=0

# Temp

In [12]:
agent = ray.rllib.agents.ppo.PPOTrainer(config, GAIAPredictorsContinuousV9)

NameError: name 'GAIAPredictorsContinuousV9' is not defined

In [33]:
env = GAIAPredictorsContinuousV9()
state = env.reset()
state

array([-0.93751576, -1.97728031, -1.        ,  0.        ,  0.        ])

In [36]:
agent.compute_action(state)

array([-0.1,  0.1])

# Restore

**Note: id 7364 stops after 2012, so 2013 had an empty prgoress cell**

In [12]:
results = WalkForwardResults('/home/Nicholas/trading-gym/notebooks/registry/gaia/v9/logs/')
results

WalkForwardResults(['GAIAPredictorsContinuousV10', 'GAIAPredictorsContinuousV9'])

In [13]:
env_results = results['GAIAPredictorsContinuousV10']
env_results

EnvResults(GAIAPredictorsContinuousV10)

In [15]:
# Step 1.
env = env_results.make_env(
    env_config={
#         'cost_of_commissions': 0.00005,
#         'cost_of_spread': 0.0001,
        'folds': {
            'training-set': [datetime.min, datetime(2008, 3, 18)],
            'test-set': [datetime(2008, 3, 19), datetime.max],
        }
    },
)

4391807037254845302 : NO LSTM high commissions, 500k steps, 0.95 gamma
-7386114885991154538: NO LSTM high commissions, 500k steps, 0.95 gamma
-1177500123347456453: NO LSTM no commissions, 500k steps, 0.95 gamma

In [16]:
env_results.restore_ids

{4391807037254845302: [AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2007-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_18-46-32hmggedyr),
  AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2009-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_19-44-55wxegjke7),
  AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2008-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_19-16-55i_vbm4yi),
  AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2013-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_21-36-18xr1gej89),
  AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2011-12-31/PPO_GAIAPr

In [17]:
restore_id = list(env_results.restore_ids)[0]
restore_id = -1177500123347456453

In [18]:
policy = env_results.make_policy(
    env=env,
    restore_id=restore_id,
    checkpoint_nr=None,  # use None (or don't specify) to use last checkpoint available
)
policy

In [19]:
episode = env.sample_episode(fold='test-set', policy=policy, verbose=False)

2019-08-24 23:17:30,288	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:17:31,941	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:17:32,202	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:

{ 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
  'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
  'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
  'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
  'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
  'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 15) dtype=float32>,
  'obs': <tf.Tensor 'default_policy/observation:0' shape=(?, 15) dtype=float32>,
  'prev_actions': <tf.Tensor

(pid=108823) 2019-08-24 23:17:40,207	INFO rollout_worker.py:301 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=108823) 2019-08-24 23:17:40.267772: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=108823) 2019-08-24 23:17:41,191	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:
(pid=108823) 
(pid=108823) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=108823)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=108823)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=108823)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
(pid=108823)   'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
(pid=108823)   'new_obs': <tf.Tensor 'default_policy/

2019-08-24 23:17:45,516	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.


(pid=92445) /home/Nicholas/trading-gym/.venv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:
(pid=92445) 
(pid=92445) Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=92445) 


2019-08-24 23:17:46,966	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:17:48,585	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:17:51,227	INFO rollout_worker.py:719 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f8d3ba13cc0>}
2019-08-24 23:17:51,232	INFO rollout_worker.py:720 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f8d3ba13898>}
2019-08-24 23:17:51,233	INFO rollout_worker.py:333 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f8d3ba136d8>}
2019-08-24 23:17:51,324	INFO multi_gpu_optimizer.py:79 -- LocalMultiGPUOptimizer devices ['/cpu:0']


...

2019-08-24 23:18:01,781	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:18:03,024	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:18:05,051	INFO rollout_worker.py:719 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f8d3340b278>}
2019-08-24 23:18:05,052	INFO rollout_worker.py:720 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f8d334aee10>}
2019-08-24 23:18:05,053	INFO rollout_worker.py:333 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f8d334aebe0>}
2019-08-24 23:18:05,141	INFO multi_gpu_optimizer.py:79 -- LocalMultiGPUOptimizer devices ['/cpu:0']


(pid=92754) 2019-08-24 23:18:08,865	INFO rollout_worker.py:301 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=92754) 2019-08-24 23:18:08.898787: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=92754) 2019-08-24 23:18:09,310	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:
(pid=92754) 
(pid=92754) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=92754)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=92754)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=92754)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
(pid=92754)   'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
(pid=92754)   'new_obs': <tf.Tensor 'default_policy/new_obs:0'

2019-08-24 23:18:15,459	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:18:19,087	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:18:21,793	INFO rollout_worker.py:719 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f8d02ec6860>}
2019-08-24 23:18:21,795	INFO rollout_worker.py:720 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f8d02ec6438>}
2019-08-24 23:18:21,797	INFO rollout_worker.py:333 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f8d02ec6208>}
2019-08-24 23:18:21,865	INFO multi_gpu_optimizer.py:79 -- LocalMultiGPUOptimizer devices ['/cpu:0']


(pid=93060) 2019-08-24 23:18:26,045	INFO rollout_worker.py:301 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=93060) 2019-08-24 23:18:26.076925: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=93060) 2019-08-24 23:18:26,341	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:
(pid=93060) 
(pid=93060) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=93060)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=93060)   'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
(pid=93060)   'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
(pid=93060)   'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
(pid=93060)   'new_obs': <tf.Tensor 'default_policy/new_obs:0'

2019-08-24 23:18:30,510	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-24 23:18:32,076	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-24 23:18:35,356	INFO rollout_worker.py:719 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f8cf2960f98>}
2019-08-24 23:18:35,358	INFO rollout_worker.py:720 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f8cf2960b70>}
2019-08-24 23:18:35,361	INFO rollout_worker.py:333 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f8cf29609b0>}
2019-08-24 23:18:35,445	INFO multi_gpu_optimizer.py:79 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-08-24 23:18:43,805	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy mod

In [20]:
episode.renderer.cumulative_performance.to_plotly()
episode.renderer.target_weights.to_plotly()
episode.renderer.annual_returns.to_plotly()
episode.renderer.tearsheet()

Strategy  \
Context              From                                       2008-03-19   
                     To                                         2018-08-28   
                     Years                                         10.4493   
                     Observations                                     2725   
                     Risk-free asset                 Index(USD 1M Deposit)   
                     Risk-free CAGR                             0.00681294   
Return               CAGR                                         0.146735   
                     CAGR over cash                               0.139922   
                     Overall return                                3.18166   
Risk                 Volatility                                   0.109967   
                     Downside volatility                         0.0815877   
                     Upside volatility                           0.0783522   
                     Max drawdown                                -0.159487   
                     Martin risk                                 0.0320039   
                     VaR 5%                                     -0.0107189   
                     VaR 2%                                      -0.015886   
                     Expected shortfall 5%                      -0.0162037   
                     Expected shortfall 2%                      -0.0211552   
Risk-adjusted return Sharpe ratio                                   1.2724   
                     Sortino ratio                                 1.71499   
                     Calmar ratio                                 0.877324   
                     Martin ratio                                  4.37203   
Outperformance       Benchmark name                  Index(Aric-Benchmark)   
                     CAGR over benchmark                        -0.0118504   
                     Information ratio                           -0.146135   
                     CAPM Alpha                                  0.0207285   
                     CAPM Beta                                    0.792677   
Weights              Cash(USD)                                  0.00368878   
                     ETF(Russell 1000, SMART, USD)                0.574251   
                     ETF(7-10Y T-Bills, SMART, USD)                0.42206   
                     Leverage mean                                       1   
                     Turnover daily                              0.0166169   
                     Turnover annual                               4.18746   

                                                     Index(Aric-Benchmark)  \
Context              From                                       2008-03-19   
                     To                                         2018-08-28   
                     Years                                         10.4493   
                     Observations                                     2725   
                     Risk-free asset                 Index(USD 1M Deposit)   
                     Risk-free CAGR                             0.00681294   
Return               CAGR                                         0.158586   
                     CAGR over cash                               0.151773   
                     Overall return                                3.65592   
Risk                 Volatility                                  0.0970738   
                     Downside volatility                          0.069906   
                     Upside volatility                           0.0705399   
                     Max drawdown                               -0.0865477   
                     Martin risk                                 0.0189993   
                     VaR 5%                                    -0.00895352   
                     VaR 2%                                     -0.0134313   
                     Expected shortfall 5%                      -0.0137858   
         

**The below bit of code is for the checkpointing/gif style images**

In [21]:
# cost2restore_id = {
#     0.00001: 1562644065721998140
# }


# cost2restore_id = {
#     0.00001: -3190398049207830876
# }

# now to find the best results for PPO with 0.00005 comiss, 500k steps, no_lstm, 1e-5 lr 
#but i've varied gamma 


# this is for v9 
gamma2restore_id = {
   0.75: 5283795639021582785,
    0.82: 847929563054400422, 
    0.85: 8716763645865485944,
    0.9:1144509479006877989  ,
    0.95: 7592239698496115625,
    0.999:-1739212663066014646
}


# now i want to do it for v10 

In [22]:
nr2episode = dict()
for gamma, restore_id in gamma2restore_id.items():
    nr2episode[gamma] = env_results.get_nr2episode(
        restore_id=restore_id,
        checkpoint_nrs=np.arange(1, 123, 1),
        fold='test-set',
        env_config={
            'folds': {
                'training-set': [datetime.min, datetime(2008, 3, 18)],
                'test-set': [datetime(2008, 3, 19), datetime.max],
            }
        }
    )

ValueError: Invalid restore_id 5283795639021582785. Valid values are
:{4391807037254845302: [AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2007-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_18-46-32hmggedyr), AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2009-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_19-44-55wxegjke7), AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2008-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_19-16-55i_vbm4yi), AgentResults(GAIAPredictorsContinuousV10_1-01-01_to_2013-12-31/PPO_GAIAPredictorsContinuousV10_restoreID=4391807037254845302_runID=LlKOdeFh_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95_2019-08-21_21-36-18xr1gej89), AgentResult

In [ ]:
nr2episode

In [ ]:
0.82: 847929563054400422, 
    0.85: 8716763645865485944,
    0.9:1144509479006877989  ,
    0.95: 7592239698496115625,
    0.999:-1739212663066014646
}

In [40]:
gamma  = 0.999
# 0.82 0.85 0.9 0.95 0.999
nr2episode[gamma].plot_weights()

interactive(children=(IntSlider(value=61, description='nr', max=183, min=-61), Output()), _dom_classes=('widge…

FigureWidget({
    'data': [{'name': 'Cash(USD)',
              'type': 'scatter',
              'uid': '49ee0…

In [41]:
nr2episode[gamma].plot_levels()

interactive(children=(IntSlider(value=61, description='nr', max=183, min=-61), Output()), _dom_classes=('widge…

FigureWidget({
    'data': [{'name': 'Strategy',
              'type': 'scatter',
              'uid': '91da6c…

In [42]:
nr2episode[gamma].plot_metrics_as_we_train()